# Transferlearning with VGG16 and a Fruit Dataset
(by: [Nicolaj Stache](mailto:Nicolaj.Stache@hs-heilbronn.de) and [Andreas Schneider](mailto:Andreas.Schneider@hs-heilbronn.de), both: Heilbronn University of Applied Sciences, Germany)

Source: https://github.com/lazyprogrammer/machine_learning_examples


### Additional dependencies:
1. *pip install matplotlib*
2. *pip install scikit-learn*

In [ ]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import tensorflow as tf

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
import zipfile
import os
import itertools

### Let's check if there is any GPU available, then training will be incredibly faster.

In [ ]:
tf.test.is_gpu_available()

### Extract the dataset and move it to the right folder

In [ ]:
# create data folder if not existing and extract data into it.
if not os.path.exists("./data"):
    os.makedirs("./data")

if not os.path.exists("./data/fruits-360-small"):    
    zip_ref = zipfile.ZipFile("fruits-360-small.zip", 'r')
    zip_ref.extractall("./data/")
    zip_ref.close()

### Define the images size and the train/val folders

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [100, 100]

train_path = 'data/fruits-360-small/Training'
valid_path = 'data/fruits-360-small/Validation'

# useful for getting number of files
image_files = glob(train_path + '/*/*.jp*g')
valid_image_files = glob(valid_path + '/*/*.jp*g')

# useful for getting number of classes
folders = glob(train_path + '/*')

### Look at a random image

In [ ]:
# look at an image for fun
plt.imshow(image.load_img(np.random.choice(image_files)))
plt.show()

### Define pre-built VGG network with imagenet weights

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

### Add layers to VGG, generate a Keras Model, double check the final structure and compile it.

In [ ]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)


# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)


### Define Hyperparameteres

In [ ]:
epochs = 3
batch_size = 16

### Create a ImageDataGenerator

Generate batches of tensor image data with real-time data augmentation. The data will be looped over (in batches).

In [ ]:
gen = ImageDataGenerator(
  rotation_range=20,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.2,
  horizontal_flip=True,
  vertical_flip=True,
  preprocessing_function=preprocess_input
)

### Create 2 seperate generators, one for training, one for validation

**flow_from_directory()**:  
*Takes the path to a directory, and generates batches of augmented/normalized data.*

In [ ]:
train_generator = gen.flow_from_directory(
  train_path,
  target_size=IMAGE_SIZE,
  shuffle=True,
  batch_size=batch_size,
)

valid_generator = gen.flow_from_directory(
  valid_path,
  target_size=IMAGE_SIZE,
  shuffle=True,
  batch_size=batch_size,
)

In [ ]:
labels = [None] * len(train_generator.class_indices)
for k, v in train_generator.class_indices.items():
    labels[v] = k
    
print(labels)

### Start Training

**Note**: We are not longer use **model.fit()**, instead we use **fit_generator()** (because of the ImageDataGenerator)  


**fit_generator():**  
Trains the model on data generated batch-by-batch by a Python generator or an instance of Sequence.
The generator is run in parallel to the model, for efficiency. For instance, this allows you to do real-time data augmentation on images on CPU in parallel to training your model on GPU.
The use of keras.utils.Sequence guarantees the ordering and guarantees the single use of every input per epoch when using use_multiprocessing=True.

In [ ]:
# fit the model
r = model.fit_generator(
    train_generator,
    validation_data=valid_generator,
    epochs=epochs,
    steps_per_epoch=len(image_files) // batch_size,
    validation_steps=len(valid_image_files) // batch_size,
    verbose=1
)

### Let's have a look at the development of Accuracy and Loss in the training process.

In [ ]:
# plot some data

# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()

### Generate a confusion matrix in order to find out which classes interfere with each other.

In [ ]:
def get_confusion_matrix(data_path, N):
    # we need to see the data in the same order
    # for both predictions and targets
    print("Generating confusion matrix", N)
    predictions = []
    targets = []
    i = 0
    for x, y in gen.flow_from_directory(data_path, target_size=IMAGE_SIZE, shuffle=False, batch_size=batch_size * 2):
        i += 1
        if i % 50 == 0:
            print(i)
        p = model.predict(x)
        p = np.argmax(p, axis=1)
        y = np.argmax(y, axis=1)
        predictions = np.concatenate((predictions, p))
        targets = np.concatenate((targets, y))
        if len(targets) >= N:
            break

    cm = confusion_matrix(targets, predictions)
    return cm


cm = get_confusion_matrix(train_path, len(image_files))
valid_cm = get_confusion_matrix(valid_path, len(valid_image_files))

### Plot the confusion matrix

In [ ]:
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Train confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)

fmt = 'd'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
           horizontalalignment="center",
           color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# ----------------

plt.imshow(valid_cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Validation confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)

fmt = 'd'
thresh = cm.max() / 2.
for i, j in itertools.product(range(valid_cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(valid_cm[i, j], fmt),
           horizontalalignment="center",
           color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()